## Notebook to extract NDVI, LandCover and Precipitation (Step 0)
Date sources: <br>
/css/modis/Collection6/L3/MOD13Q1-Vegetation
/css/modis/Collection6/L3/MCD12Q1-Landcover
/css/imerg/daily_Late_V06

In [1]:
import os
import glob
import datetime
from osgeo import gdal
import numpy as np

In [2]:
def hdf_sds_to_tif(in_hdf, out_tif, key="NDVI"):
    # input hdf (absolute path) 
    # extract subdataset and export as geoTiff

    hdf = gdal.Open(in_hdf)
    sdsdict = hdf.GetMetadata("SUBDATASETS")
    sdslist =[sdsdict[k] for k in sdsdict.keys() if '_NAME' in k]
    
    #search for target layer in hdf
    layer = [i for i in sdslist if key in i]
    
    # process 
    if layer:
        cmd = f"gdal_translate {layer[0]} {out_tif}"
        os.system(cmd)
    else:
        raise RuntimeError(f'Can not find {key} in {in_hdf}')

In [3]:
# dirs of input datasets
ndvi_dirn = "/css/modis/Collection6/L3/MOD13Q1-Vegetation"
lc_dirn = "/css/modis/Collection6/L3/MCD12Q1-Landcover"
pr_dirn = "/css/imerg/daily_Late_V06"

In [4]:
# dirs of output datasets
ndvi_out_dirn = "/att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/NDVI"
pr_out_dirn = "/att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP"
lc_out_dirn = "/att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/LANDCOVER"

# create dirs if not exist
if not os.path.exists(ndvi_out_dirn):
    os.mkdir(ndvi_out_dirn)
if not os.path.exists(pr_out_dirn):
    os.mkdir(pr_out_dirn)    
if not os.path.exists(lc_out_dirn):
    os.mkdir(lc_out_dirn)

In [5]:
year = 2006
tile = "h16v07" # target tile index
days = np.arange(1,365,16) # MOD13 freq 

vihis = 5 # past 5 years NDVI
tint = 30 # past 30 days Precip

In [6]:
days[5:6]

array([81])

In [7]:
%%time
#loop through days
for day in days[5:6]:
    
    yj = str(year)+str(day).zfill(3)
    c_date = datetime.datetime.strptime(yj, "%Y%j")
    print("Current Date: ", c_date)
    
    #--------------------------------------------------------------------------
    # Extract NDVI
    #--------------------------------------------------------------------------
    # form a list containing current and past 5-yr MOD13-veg files
    ndvi_date_p = [c_date.replace(year=c_date.year-i) for i in range(vihis+1)]   
    ndvi_f_list = []
    for d in ndvi_date_p:
        yyyy = str(d.year)
        ddd = str(day).zfill(3)
        path = os.path.join(ndvi_dirn,
                           yyyy,
                           ddd,
                           f"*{tile}*.hdf")
        f = glob.glob(path)
        ndvi_f_list.append(f[0])
    # convert subdataset to geoTiff
    for f in ndvi_f_list:
        fn = os.path.basename(f)
        fn = os.path.splitext(fn)[0]+".ndvi.tif"
        ofile = os.path.join(ndvi_out_dirn, fn)
    
        if not os.path.exists(ofile):
            hdf_sds_to_tif(f, ofile)
    #--------------------------------------------------------------------------
    # Extract Precip
    #--------------------------------------------------------------------------
    # for a list containing current and past 30-day precip files
    pr_date_p = [c_date-datetime.timedelta(days=i) for i in range(tint+1)]
    pr_f_list = []
    for d in pr_date_p:
        yyyy = str(d.year)
        tstamp = datetime.datetime.strftime(d, "%Y%m%d")
        path = os.path.join(pr_dirn,
                           yyyy,
                           f"*.{tstamp}*.nc4")
        f = glob.glob(path)
        pr_f_list.append(f[0])
    # convert subdataset to geoTiff
    for f in pr_f_list:
        fn = os.path.basename(f)
        fn = os.path.splitext(fn)[0]+".precip.tif"
        ofile = os.path.join(pr_out_dirn, fn)
    
        if not os.path.exists(ofile):
            hdf_sds_to_tif(f, ofile, key="precipitationCal")
    
    #--------------------------------------------------------------------------
    # Extract LandCover
    #--------------------------------------------------------------------------
    # LandCover 1 file per year
    path = os.path.join(lc_dirn,
                   str(year),
                   "001",
                   f"*{str(year)}001*{tile}*.hdf")
    f = glob.glob(path)[0]
    
    # convert subdataset to geoTiff
    fn = os.path.basename(f)
    fn = os.path.splitext(fn)[0]+".landcover.tif"        
    ofile = os.path.join(lc_out_dirn, fn)
    
    if not os.path.exists(ofile):
        hdf_sds_to_tif(f, ofile, key="LC_Type1")

Current Date:  2006-03-22 00:00:00
Input file size is 4800, 4800
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4800, 4800
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4800, 4800
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4800, 4800
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4800, 4800
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4800, 4800
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1800, 3600
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1800, 3600
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1800, 3600
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1800, 3600
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1800, 3600
0...10...20...30...40...5